In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
file_name = 'https://drive.google.com/file/d/1s3x8sABy2JfEoSHE-VsJeziuktFSPpon/view?usp=drive_link'

In [3]:
!pip install super-gradients

In [4]:
import super_gradients
yolo_nas = super_gradients.training.models.get("yolo_nas_pose_l", pretrained_weights="coco_pose")

[2024-05-29 07:15:46] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log
[WARNING]No module named 'pycocotools'


[2024-05-29 07:15:53] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco_pose dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-05-29 07:15:53] INFO - checkpoint_utils.py - License Notification: YOLO-NAS-POSE pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS-POSE.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-05-29 07:15:54] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_pose_l


In [5]:
import cv2
import time
import math as m

In [6]:
# Calculate angle with vector approach
def findAngle(x1, y1, x2, y2):
    #since y3 is valid for all y, let's take y3=0 for simplicity.
    theta = m.acos((y2 -y1)*(-y1) / (m.sqrt((x2 - x1)**2 + (y2 - y1)**2) * y1))
    degree = int(180/m.pi)*theta
    return degree

In [7]:
# Initilize frame counters.
good_frames = 0
bad_frames = 0

# Font type.
font = cv2.FONT_HERSHEY_SIMPLEX

# Colors.
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

In [8]:
# Initialize video capture object.
file_name = '/content/chair_u.mp4'
cap = cv2.VideoCapture(file_name)

# Meta.
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Initialize video writer.
video_output = cv2.VideoWriter('output.mp4', fourcc, fps, frame_size)

In [9]:
import math

def calculate_angle(x1, y1, x2, y2, x3, y3):

  # Calculate the vector lengths for each arm segment
  upper_arm_length = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
  lower_arm_length = math.sqrt((x3 - x2)**2 + (y3 - y2)**2)

  # Ensure we don't have division by zero (shoulders and elbow can't coincide)
  if upper_arm_length == 0:
    return 0

  # Calculate the dot product of the normalized arm segment vectors
  dot_product = ((x2 - x1) / upper_arm_length) * ((x3 - x2) / lower_arm_length) + \
                ((y2 - y1) / upper_arm_length) * ((y3 - y2) / lower_arm_length)

  # Apply the law of cosines to find the angle
  angle_rad = math.acos(dot_product)

  # Convert from radians to degrees
  angle_deg = int(180 / math.pi) * angle_rad

  return angle_deg


In [10]:
print('Processing..')
while cap.isOpened():
    # Capture frames.
    success, image = cap.read()
    if not success:
        print("Null.Frames")
        break
    # Get fps.
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Get height and width.
    h, w = image.shape[:2]

    # Convert the BGR image to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image.
    predictions = yolo_nas.predict(image, conf=0.5)

    # Convert the image back to BGR.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Acquire the landmark coordinates.
    # Once aligned properly, left or right should not be a concern.
    # Left shoulder.
    poses = predictions.prediction.poses  # Assuming 'prediction' has a nested 'prediction' attribute

  # Loop through each pose
    for pose in poses:
    # Access individual keypoints within the pose
      for i, keypoint in enumerate(pose):

        if i == 4:
          r_ear_x = int(keypoint[0])  # Assuming x is the first element
          r_ear_y = int(keypoint[1])  # Assuming y is the second element

        elif i == 6:
          r_shldr_x = int(keypoint[0])  # Assuming x is the first element
          r_shldr_y = int(keypoint[1])  # Assuming y is the second element

        elif i == 8:
          r_elbow_x = int(keypoint[0])  # Assuming x is the first element
          r_elbow_y = int(keypoint[1])  # Assuming y is the second element

        elif i == 10:
          r_wrist_x = int(keypoint[0])  # Assuming x is the first element
          r_wrist_y = int(keypoint[1])  # Assuming y is the second element

        elif i == 12:
          r_hip_x = int(keypoint[0])  # Assuming x is the first element
          r_hip_y = int(keypoint[1])  # Assuming y is the second element

        elif i == 14:
          r_knee_x = int(keypoint[0])  # Assuming x is the first element
          r_knee_y = int(keypoint[1])  # Assuming y is the second element

        elif i == 16:
          r_ancle_x = int(keypoint[0])  # Assuming x is the first element
          r_ancle_y = int(keypoint[1])  # Assuming y is the second element


    # Calculate angles.
    neck_inclination = findAngle(r_shldr_x, r_shldr_y, r_ear_x, r_ear_y)
    torso_inclination = findAngle(r_hip_x, r_hip_y, r_shldr_x, r_shldr_y)

    knee_inclination = calculate_angle(r_hip_x, r_hip_y, r_knee_x, r_knee_y, r_ancle_x, r_ancle_y)
    arm_inclination = calculate_angle(r_shldr_x, r_shldr_y, r_elbow_x, r_elbow_y, r_wrist_x, r_wrist_y)

    angle_text_string = 'Neck : ' + str(int(neck_inclination)) + ' Torso : ' + str(int(torso_inclination))

    # Draw landmarks.
    cv2.circle(image, (r_shldr_x, r_shldr_y), 7, yellow, -1)
    cv2.circle(image, (r_ear_x, r_ear_y), 7, yellow, -1)

    cv2.circle(image, (r_shldr_x, r_shldr_y - 100), 7, yellow, -1)
    cv2.circle(image, (r_shldr_x, r_shldr_y), 7, pink, -1)

    cv2.circle(image, (r_hip_x, r_hip_y), 7, yellow, -1)
    cv2.circle(image, (r_hip_x, r_hip_y - 100), 7, yellow, -1)

    cv2.circle(image, (r_elbow_x, r_elbow_y), 7, yellow, -1)
    cv2.circle(image, (r_knee_x, r_knee_y), 7, yellow, -1)


    cv2.circle(image, (r_ancle_x, r_ancle_y), 7, yellow, -1)
    cv2.circle(image, (r_wrist_x, r_wrist_y), 7, yellow, -1)


    if neck_inclination < 40 and torso_inclination < 10 :

        cv2.putText(image, angle_text_string, (10, 30), font, 0.9, light_green, 2)


        # Join landmarks.
        cv2.line(image, (r_shldr_x, r_shldr_y), (r_ear_x, r_ear_y), green, 4)
        cv2.line(image, (r_shldr_x, r_shldr_y), (r_shldr_x, r_shldr_y - 100), green, 4)

        cv2.line(image, (r_hip_x, r_hip_y), (r_shldr_x, r_shldr_y), green, 4)
        cv2.line(image, (r_hip_x, r_hip_y), (r_hip_x, r_hip_y - 100), green, 4)

    elif 90 < knee_inclination < 100:

        cv2.putText(image, str(knee_inclination), (r_knee_x + 5 , r_knee_y + 5), font, 0.9, light_green, 2)

        cv2.line(image, (r_knee_x, r_knee_y), (r_ancle_x, r_ancle_y), green, 4)
        cv2.line(image, (r_hip_x, r_hip_y), (r_knee_x, r_knee_y), green, 4)

    elif 90 < arm_inclination < 100:

        cv2.putText(image, str(arm_inclination), (r_knee_x + 5 , r_knee_y + 5), font, 0.9, light_green, 2)

        cv2.line(image,  (r_elbow_x, r_elbow_y), (r_wrist_x, r_wrist_y), green, 4)
        cv2.line(image,  (r_shldr_x, r_shldr_y), (r_elbow_x, r_elbow_y), green, 4)

    else:

        cv2.putText(image, angle_text_string, (10, 30), font, 0.9, red, 2)
        cv2.putText(image, str(knee_inclination), (r_knee_x + 5 , r_knee_y + 5), font, 0.9, red, 2)
        cv2.putText(image, str(arm_inclination), (r_knee_x + 5 , r_knee_y + 5), font, 0.9, red, 2)


        # Join landmarks.
        cv2.line(image, (r_shldr_x, r_shldr_y), (r_ear_x, r_ear_y), red, 4)
        cv2.line(image, (r_shldr_x, r_shldr_y), (r_shldr_x, r_shldr_y - 100), red, 4)

        cv2.line(image, (r_hip_x, r_hip_y), (r_shldr_x, r_shldr_y), red, 4)
        cv2.line(image, (r_hip_x, r_hip_y), (r_hip_x, r_hip_y - 100), red, 4)

        cv2.line(image, (r_knee_x, r_knee_y), (r_ancle_x, r_ancle_y), red, 4)
        cv2.line(image, (r_hip_x, r_hip_y), (r_knee_x, r_knee_y), red, 4)

        cv2.line(image,  (r_elbow_x, r_elbow_y), (r_wrist_x, r_wrist_y), red, 4)
        cv2.line(image,  (r_shldr_x, r_shldr_y), (r_elbow_x, r_elbow_y), red, 4)


    video_output.write(image)

cap.release()
video_output.release()